In [1]:
# Step2 최종 입지 변수 결합 (Step1 리뷰 통합 포함)
# =========================================
import pandas as pd
import numpy as np
import re

In [2]:
# 0. Step1 전처리 CSV 불러오기 (리뷰 수 + 로그 컬럼 포함)
# =========================================
df_step1 = pd.read_csv(
    r"C:\Users\user\Desktop\데이터분석\분석용\Step1_전처리_자사경쟁사.csv",
    encoding="utf-8-sig"
)

In [3]:
# 1. 점포별 CSV 2개 불러오기 & 합병 (브랜드 기본 정보)
# =========================================
files = [
    r"C:\Users\user\Desktop\데이터분석\정제\금별맥주_정제완료.csv",
    r"C:\Users\user\Desktop\데이터분석\정제\역전할머니_정제완료.csv"
]
df_list = [pd.read_csv(f, encoding='utf-8') for f in files]
요리주점합병 = pd.concat(df_list, ignore_index=True)

In [4]:
# 2. 프랜차이즈 여부 컬럼 생성
# =========================================
요리주점합병["프랜차이즈"] = np.where(
    요리주점합병["brand"].isin(["금별맥주", "역전할머니"]),
    1,
    0
)

In [5]:
# 3. 브랜드별 출구 기준 도보거리 CSV 불러오기 & 합병
# =========================================
subway_files = [
    r"C:\Users\user\Desktop\데이터분석\수집 후 출력(CSV)\금별_역과의거리.csv",
    r"C:\Users\user\Desktop\데이터분석\수집 후 출력(CSV)\역전_역과의거리.csv"
]
subway_list = [pd.read_csv(f, encoding='utf-8') for f in subway_files]
최근접지하철역합병 = pd.concat(subway_list, ignore_index=True)

In [6]:
# 4. 시군구 기준 요리주점 수 및 프랜차이즈 비율 계산
# =========================================
시군구_요리주점 = 요리주점합병.groupby("시군구명").size().reset_index(name="시군구_요리주점_수")
시군구_프랜차이즈 = 요리주점합병.groupby("시군구명")["프랜차이즈"].mean().reset_index(name="프랜차이즈_비율")

In [7]:
# 5. 시군구 면적 데이터 불러오기 (서울 + 경기)
# =========================================
df_면적_서울 = pd.read_csv(r"C:\Users\user\Desktop\데이터분석\정제\4.2_시군구_서울_면적.csv")
df_면적_경기 = pd.read_csv(r"C:\Users\user\Desktop\데이터분석\정제\4.2_시군구_경기_면적.csv")

df_면적_서울 = df_면적_서울.rename(columns={"시군구": "시군구명", "면적_km2": "면적_km2"})
df_면적_경기 = df_면적_경기.rename(columns={"시군구": "시군구명", "면적_km2": "면적_km2"})

시군구_면적 = pd.concat([df_면적_서울, df_면적_경기], ignore_index=True)[["시군구명", "면적_km2"]]

In [8]:
# 6. 시군구 단위 요약 테이블 생성 (밀도 계산)
# =========================================
시군구_요약 = (
    시군구_요리주점
    .merge(시군구_프랜차이즈, on="시군구명", how="left")
    .merge(시군구_면적, on="시군구명", how="left")
)
시군구_요약["요리주점_밀도"] = 시군구_요약["시군구_요리주점_수"] / 시군구_요약["면적_km2"]

In [9]:
# 7. 출구 기준 도보거리 처리 (근접/비근접 400m 기준)
# =========================================
def extract_distance(text):
    if pd.isna(text):
        return np.nan
    nums = re.findall(r"\d+\.?\d*", str(text))
    return float(nums[0]) if nums else np.nan

최근접지하철역합병["출구_도보거리_m"] = 최근접지하철역합병["exit_info"].apply(extract_distance)
최근접지하철역합병["역_거리_구분"] = np.where(
    최근접지하철역합병["출구_도보거리_m"] <= 400,
    "근접",
    "비근접"
)

In [10]:
# 8. 점포 데이터와 Step1 리뷰 + 시군구 요약 + 출구 거리 병합
# =========================================
최종_입지_병합 = pd.merge(
    요리주점합병,
    df_step1[["place_id", "리뷰수", "방문자_리뷰수", "블로그_리뷰수", "log_리뷰수", "log_방문자_리뷰수", "log_블로그_리뷰수"]],
    on="place_id",
    how="left"
)
최종_입지_병합 = pd.merge(
    최종_입지_병합,
    최근접지하철역합병[["place_id", "출구_도보거리_m", "역_거리_구분"]],
    on="place_id",
    how="left"
)
최종_입지_병합 = pd.merge(
    최종_입지_병합,
    시군구_요약[["시군구명", "요리주점_밀도", "프랜차이즈_비율"]],
    on="시군구명",
    how="left"
)

In [12]:
# 9. 최종 CSV 저장
# =========================================
최종_입지_병합.to_csv(
    r"C:\Users\user\Desktop\데이터분석\분석용\Step2_입지변수_결합완료.csv",
    index=False,
    encoding="utf-8-sig"
)

In [14]:
# 10. 핵심 컬럼 미리보기
# =========================================
핵심컬럼 = [
    "place_id", "지점명", "brand", "프랜차이즈",
    "시군구명", "시군구_요리주점_수", "면적_km2", "요리주점_밀도", "프랜차이즈_비율",
    "출구_도보거리_m", "역_거리_구분",
    "리뷰수", "방문자_리뷰수", "블로그_리뷰수",
    "log_리뷰수", "log_방문자_리뷰수", "log_블로그_리뷰수"
]
existing_cols = [c for c in 핵심컬럼 if c in 최종_입지_병합.columns]
print(최종_입지_병합[existing_cols].head())

     place_id     지점명 brand  프랜차이즈  시군구명   요리주점_밀도  프랜차이즈_비율  출구_도보거리_m  \
0  1113384749  동양미래대점  금별맥주      1   구로구  0.248509       1.0        2.0   
1  1320051621   양천향교점  금별맥주      1   강서구  0.144770       1.0        8.0   
2  1588254549     NaN  금별맥주      1   송파구  0.103306       1.0        4.0   
3  1601981946    송정역점  금별맥주      1   강서구  0.144770       1.0        4.0   
4  1161335521     NaN  금별맥주      1  서대문구  0.141804       1.0        2.0   

  역_거리_구분  리뷰수  방문자_리뷰수  블로그_리뷰수   log_리뷰수  log_방문자_리뷰수  log_블로그_리뷰수  
0      근접  160      267     58.0  5.081404     5.590987     4.077537  
1      근접  218      244     47.0  5.389072     5.501258     3.871201  
2      근접  158      234     42.0  5.068904     5.459586     3.761200  
3      근접  169      240     24.0  5.135798     5.484797     3.218876  
4      근접  413      505    712.0  6.025866     6.226537     6.569481  


In [15]:
# 11. 근접/비근접 점포 수 확인
근접_비근접_갯수 = 최종_입지_병합["역_거리_구분"].value_counts()
print("전체 점포 기준 근접/비근접 점포 수:")
print(근접_비근접_갯수)

브랜드별_근접_비근접 = 최종_입지_병합.groupby("brand")["역_거리_구분"].value_counts().unstack(fill_value=0)
print("\n브랜드별 근접/비근접 점포 수:")
print(브랜드별_근접_비근접)

전체 점포 기준 근접/비근접 점포 수:
역_거리_구분
근접     269
비근접    161
Name: count, dtype: int64

브랜드별 근접/비근접 점포 수:
역_거리_구분   근접  비근접
brand            
금별맥주      56   47
역전할머니    213  114
